# Tutorial 1: Basics of CrypTen Tensors

We now have a high-level understanding of how secure MPC works. Through these tutorials, we will explain how to use CrypTen to carry out secure operations on encrypted tensors. In this tutorial, we will introduce a fundamental building block in CrypTen, called a ```CrypTensor```.  ```CrypTensors``` are encrypted ```torch``` tensors that can be used for computing securely on data. 

CrypTen currently only supports secure MPC protocols (though we intend to add support for other advanced encryption protocols). Using the ```mpc``` backend, ```CrypTensors``` act as ```torch``` tensors whose values are encrypted using secure MPC protocols. Tensors created using the ```mpc``` backend are called ```MPCTensors```. We will go into greater detail about ```MPCTensors``` in Tutorial 2. 

Let's begin by importing ```crypten``` and ```torch``` libraries. (If the imports fail, please see the installation instructions in the README.)

In [1]:
import crypten
import torch

crypten.init()

/Users/memo/.pyenv/versions/3.11.4/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TESTING


In [1]:
import crypten
import torch

crypten.init()

torch.set_printoptions(sci_mode=False)

#Construct example input CrypTensor
x = torch.tensor([-0.1, -0.3, -0.5, -1.0, -1.5, -2.0, -2.5])
x_enc = crypten.cryptensor(x)

# Exp
z = x.exp()          # Public
z_enc = x_enc.exp()  # Private
print("\nPublic  exponential:", z)
# print("Private exponential:", z_enc / 2**16)
print("Private exponential:", z_enc.get_plain_text())


# Public  logarithm: tensor([-2.3026, -1.2040, -0.6931,  0.0000,  0.4055,  0.6931,  0.9163])
# Haar Pr logarithm: tensor([-2.2880, -1.1886, -0.6777,  0.0078,  0.4107,  0.6970,  0.9194])
# Private logarithm: tensor([-2.2130, -1.1631, -0.6624,  0.0155,  0.4158,  0.7009,  0.9225])

# Public  reciprocal: tensor([10.0000,  3.3333,  2.0000,  1.0000, -0.6667,  0.5000,  0.4000])
# LUT PR reciprocal: tensor([ 9.9895,  3.3300,  1.9980,  0.9995, -0.6664,  0.4999,  0.3999])
# Private reciprocal: tensor([10.0009,  3.3335,  2.0000,  1.0000, -0.6667,  0.5000,  0.4000])
# Private reciprocal: tensor([ 9.8652,  3.3016,  1.9897,  1.0000, -0.6675,  0.4993,  0.3993])

"""
LUTS vs. Approximations

Worse in sqrt, inv sqrt, reciprocal

Public  reciprocal: tensor([10.0000,  3.3333,  2.0000,  1.0000, -0.6667,  0.5000,  0.4000])
Private reciprocal: tensor([ 9.9895,  3.3300,  1.9980,  0.9995, -0.6664,  0.4999,  0.3999])
Private reciprocal: tensor([10.0009,  3.3335,  2.0000,  1.0000, -0.6667,  0.5000,  0.4000])

Public  logarithm: tensor([-2.3026, -1.2040, -0.6931,  0.0000,     nan,  0.6931,  0.9163])
Private logarithm: tensor([-2.3015, -1.2030, -0.6922,  0.0005,  2.6742,  0.6934,  0.9165])
Private logarithm: tensor([-2.3181, -1.2110, -0.6997,  0.0004, 2673119232., 0.6918, 0.9150])

Public  exponential: tensor([ 1.1052,  1.3499,  1.6487,  2.7183,  0.2231,  7.3891, 12.1825])
Private exponential: tensor([ 1.1053,  1.3503,  1.6495,  2.7196, 1983742.8750, 7.3927, 12.1885])
Private exponential: tensor([ 1.1021,  1.3440,  1.6468,  2.7121,  0.2217,  7.3280, 12.0188])

Public  square root: tensor([0.3162, 0.5477, 0.7071, 1.0000,    nan, 1.4142, 1.5811])
Private square root: tensor([0.3164, 0.5480, 0.7074, 1.0002, 3.8079, 1.4144, 1.5813])
Private square root: tensor([0.3147, 0.5477, 0.7071, 0.9989, 395268704.0000, 1.4141, 1.5811])

Public  inv square root: tensor([3.1623, 1.8257, 1.4142, 1.0000,    nan, 0.7071, 0.6325])
Private inv square root: tensor([3.1606, 1.8248, 1.4135, 0.9997, 0.2626, 0.7070, 0.6324])
Private inv square root: tensor([3.1473, 1.8258, 1.4142, 0.9989, -263516496.0000, 0.7070, 0.6324])

Public  tanh: tensor([ 0.0997,  0.2913,  0.4621,  0.7616, -0.9051,  0.9640,  0.9866])
Private tanh: tensor([ 0.0998,  0.2916,  0.4625,  0.7618, 1.0000,   0.9641,  0.9866])
Private tanh: tensor([ 0.0994,  0.2914,  0.4636,  0.7636, -0.9069,  0.9652,  0.9873])

Public  sin: tensor([ 0.0998,  0.2955,  0.4794,  0.8415, -0.9975,  0.9093,  0.5985])
Private sin: tensor([0.0999, 0.2958, 0.4799, 0.8417, 0.9347, 0.9091, 0.5981])
Private sin: tensor([ 0.0935,  0.2921,  0.4786,  0.8418, -1.0027,  0.9164,  0.6093])

Public  cos: tensor([ 0.9950,  0.9553,  0.8776,  0.5403,  0.0707, -0.4161, -0.8011])
Private cos: tensor([ 0.9950,  0.9552,  0.8773,  0.5399, -0.3554, -0.4166, -0.8014])
Private cos: tensor([ 0.9960,  0.9576,  0.8794,  0.5443,  0.0739, -0.4123, -0.8051])

Public  erf: tensor([ 0.1125,  0.3286,  0.5205,  0.8427, -0.9661,  0.9953,  0.9996])
Private erf: tensor([ 0.1126,  0.3289,  0.5209,  0.8429, 1.0000,   0.9953,  0.9996])
Private erf: tensor([ 0.1125,  0.3284,  0.5189,  0.8427, 1.5027, -175.4813, -10290.0742])



Public  exponential: tensor([0.9048, 0.7408, 0.6065, 0.3679, 0.2231, 0.1353, 0.0821])
Private exponential: tensor([0.8499, 0.6959, 0.5696, 0.3455, 0.2095, 0.1271, 0.0771]) - LUT
Private exponential: tensor([0.9066, 0.7423, 0.6054, 0.3665, 0.2217, 0.1340, 0.0809])
"""


/Users/memo/.pyenv/versions/3.11.4/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Public  exponential: tensor([0.9048, 0.7408, 0.6065, 0.3679, 0.2231, 0.1353, 0.0821])
Private exponential: tensor([0.8487, 0.6954, 0.5676, 0.3442, 0.2088, 0.1266, 0.0768])


'\nLUTS vs. Approximations\n\nWorse in sqrt, inv sqrt, reciprocal\n\nPublic  reciprocal: tensor([10.0000,  3.3333,  2.0000,  1.0000, -0.6667,  0.5000,  0.4000])\nPrivate reciprocal: tensor([ 9.9895,  3.3300,  1.9980,  0.9995, -0.6664,  0.4999,  0.3999])\nPrivate reciprocal: tensor([10.0009,  3.3335,  2.0000,  1.0000, -0.6667,  0.5000,  0.4000])\n\nPublic  logarithm: tensor([-2.3026, -1.2040, -0.6931,  0.0000,     nan,  0.6931,  0.9163])\nPrivate logarithm: tensor([-2.3015, -1.2030, -0.6922,  0.0005,  2.6742,  0.6934,  0.9165])\nPrivate logarithm: tensor([-2.3181, -1.2110, -0.6997,  0.0004, 2673119232., 0.6918, 0.9150])\n\nPublic  exponential: tensor([ 1.1052,  1.3499,  1.6487,  2.7183,  0.2231,  7.3891, 12.1825])\nPrivate exponential: tensor([ 1.1053,  1.3503,  1.6495,  2.7196, 1983742.8750, 7.3927, 12.1885])\nPrivate exponential: tensor([ 1.1021,  1.3440,  1.6468,  2.7121,  0.2217,  7.3280, 12.0188])\n\nPublic  square root: tensor([0.3162, 0.5477, 0.7071, 1.0000,    nan, 1.4142, 1.581

### Creating Encrypted Tensors
CrypTen provides a ```crypten.cryptensor``` factory function, similar to ```torch.tensor```, to make creating ```CrypTensors``` easy. 

Let's begin by creating a ```torch``` tensor and encrypting it using ```crypten.cryptensor```. To decrypt a ```CrypTensor```, use ```get_plain_text()``` to return the original tensor.  (```CrypTensors``` can also be created directly from a list or an array.)


In [2]:
# Create torch tensor
x = torch.tensor([1.0, 2.0, 3.0])

# Encrypt x
x_enc = crypten.cryptensor(x)

# Decrypt x
x_dec = x_enc.get_plain_text()   
crypten.print(x_dec)


# Create python list
y = [4.0, 5.0, 6.0]

# Encrypt x
y_enc = crypten.cryptensor(y)

# Decrypt x
y_dec = y_enc.get_plain_text()
crypten.print(y_dec)

tensor([1., 2., 3.])
tensor([4., 5., 6.])


## Operations on Encrypted Tensors
Now let's look at what we can do with our ```CrypTensors```.

#### Arithmetic Operations
We can carry out regular arithmetic operations between ```CrypTensors```, as well as between ```CrypTensors``` and plaintext tensors. Note that these operations never reveal any information about encrypted tensors (internally or externally) and return an encrypted tensor output.

In [3]:
#Arithmetic operations between CrypTensors and plaintext tensors
x_enc = crypten.cryptensor([1.0, 2.0, 3.0])

y = 2.0
y_enc = crypten.cryptensor(2.0)


# Addition
z_enc1 = x_enc + y      # Public
z_enc2 = x_enc + y_enc  # Private
crypten.print("\nPublic  addition:", z_enc1.get_plain_text())
crypten.print("Private addition:", z_enc2.get_plain_text())


# Subtraction
z_enc1 = x_enc - y      # Public
z_enc2 = x_enc - y_enc  # Private
crypten.print("\nPublic  subtraction:", z_enc1.get_plain_text())
print("Private subtraction:", z_enc2.get_plain_text())

# Multiplication
z_enc1 = x_enc * y      # Public
z_enc2 = x_enc * y_enc  # Private
print("\nPublic  multiplication:", z_enc1.get_plain_text())
print("Private multiplication:", z_enc2.get_plain_text())

# Division
z_enc1 = x_enc / y      # Public
z_enc2 = x_enc / y_enc  # Private
print("\nPublic  division:", z_enc1.get_plain_text())
print("Private division:", z_enc2.get_plain_text())


Public  addition: tensor([3., 4., 5.])
Private addition: tensor([3., 4., 5.])

Public  subtraction: tensor([-1.,  0.,  1.])
Private subtraction: tensor([-1.,  0.,  1.])

Public  multiplication: tensor([2., 4., 6.])
Private multiplication: tensor([2., 4., 6.])

Public  division: tensor([0.5000, 1.0000, 1.5000])
Private division: tensor([0.5000, 1.0000, 1.5000])


#### Comparisons
Similarly, we can compute element-wise comparisons on ```CrypTensors```. Like arithmetic operations, comparisons performed on ```CrypTensor```s will return a ```CrypTensor``` result. Decrypting these result ```CrypTensor```s will evaluate to 0's and 1's corresponding to ```False``` and ```True``` values respectively.

In [4]:
#Construct two example CrypTensors
x_enc = crypten.cryptensor([1.0, 2.0, 3.0, 4.0, 5.0])

y = torch.tensor([5.0, 4.0, 3.0, 2.0, 1.0])
y_enc = crypten.cryptensor(y)

# Print values:
print("x: ", x_enc.get_plain_text())
print("y: ", y_enc.get_plain_text())

# Less than
z_enc1 = x_enc < y      # Public
z_enc2 = x_enc < y_enc  # Private
print("\nPublic  (x < y) :", z_enc1.get_plain_text())
print("Private (x < y) :", z_enc2.get_plain_text())

# Less than or equal
z_enc1 = x_enc <= y      # Public
z_enc2 = x_enc <= y_enc  # Private
print("\nPublic  (x <= y):", z_enc1.get_plain_text())
print("Private (x <= y):", z_enc2.get_plain_text())

# Greater than
z_enc1 = x_enc > y      # Public
z_enc2 = x_enc > y_enc  # Private
print("\nPublic  (x > y) :", z_enc1.get_plain_text())
print("Private (x > y) :", z_enc2.get_plain_text())

# Greater than or equal
z_enc1 = x_enc >= y      # Public
z_enc2 = x_enc >= y_enc  # Private
print("\nPublic  (x >= y):", z_enc1.get_plain_text())
print("Private (x >= y):", z_enc2.get_plain_text())

# Equal
z_enc1 = x_enc == y      # Public
z_enc2 = x_enc == y_enc  # Private
print("\nPublic  (x == y):", z_enc1.get_plain_text())
print("Private (x == y):", z_enc2.get_plain_text())

# Not Equal
z_enc1 = x_enc != y      # Public
z_enc2 = x_enc != y_enc  # Private
print("\nPublic  (x != y):", z_enc1.get_plain_text())
print("Private (x != y):", z_enc2.get_plain_text())


x:  tensor([1., 2., 3., 4., 5.])
y:  tensor([5., 4., 3., 2., 1.])

Public  (x < y) : tensor([1., 1., 0., 0., 0.])
Private (x < y) : tensor([1., 1., 0., 0., 0.])

Public  (x <= y): tensor([1., 1., 1., 0., 0.])
Private (x <= y): tensor([1., 1., 1., 0., 0.])

Public  (x > y) : tensor([0., 0., 0., 1., 1.])
Private (x > y) : tensor([0., 0., 0., 1., 1.])

Public  (x >= y): tensor([0., 0., 1., 1., 1.])
Private (x >= y): tensor([0., 0., 1., 1., 1.])

Public  (x == y): tensor([0., 0., 1., 0., 0.])
Private (x == y): tensor([0., 0., 1., 0., 0.])

Public  (x != y): tensor([1., 1., 0., 1., 1.])
Private (x != y): tensor([1., 1., 0., 1., 1.])


#### Advanced mathematics
We are also able to compute more advanced mathematical functions on ```CrypTensors``` using iterative approximations. CrypTen provides MPC support for functions like reciprocal, exponential, logarithm, square root, tanh, etc. Notice that these are subject to numerical error due to the approximations used. 

Additionally, note that some of these functions will fail silently when input values are outside of the range of convergence for the approximations used. These do not produce errors because value are encrypted and cannot be checked without decryption. Exercise caution when using these functions. (It is good practice here to normalize input values for certain models.)

In [2]:
torch.set_printoptions(sci_mode=False)

#Construct example input CrypTensor
x = torch.tensor([0.1, 0.3, 0.5, 1.0, 1.5, 2.0, 2.5])
x_enc = crypten.cryptensor(x)

# Reciprocal
z = x.reciprocal()          # Public
z_enc = x_enc.reciprocal()  # Private
print("\nPublic  reciprocal:", z)
print("Private reciprocal:", z_enc.get_plain_text())

# Logarithm
z = x.log()          # Public
z_enc = x_enc.log()  # Private
print("\nPublic  logarithm:", z)
print("Private logarithm:", z_enc.get_plain_text())

# Exp
z = x.exp()          # Public
z_enc = x_enc.exp()  # Private
print("\nPublic  exponential:", z)
print("Private exponential:", z_enc.get_plain_text())

# Sqrt
z = x.sqrt()          # Public
z_enc = x_enc.sqrt()  # Private
print("\nPublic  square root:", z)
print("Private square root:", z_enc.get_plain_text())

# Tanh
z = x.tanh()          # Public
z_enc = x_enc.tanh()  # Private
print("\nPublic  tanh:", z)
print("Private tanh:", z_enc.get_plain_text())



Public  reciprocal: tensor([10.0000,  3.3333,  2.0000,  1.0000,  0.6667,  0.5000,  0.4000])
Private reciprocal: tensor([10.0009,  3.3335,  2.0000,  1.0000,  0.6667,  0.5000,  0.4000])

Public  logarithm: tensor([-2.3026, -1.2040, -0.6931,  0.0000,  0.4055,  0.6931,  0.9163])
Private logarithm: tensor([    -2.3181,     -1.2110,     -0.6997,      0.0004,      0.4038,
             0.6918,      0.9150])

Public  exponential: tensor([ 1.1052,  1.3499,  1.6487,  2.7183,  4.4817,  7.3891, 12.1825])
Private exponential: tensor([ 1.1021,  1.3440,  1.6468,  2.7121,  4.4574,  7.3280, 12.0188])

Public  square root: tensor([0.3162, 0.5477, 0.7071, 1.0000, 1.2247, 1.4142, 1.5811])
Private square root: tensor([0.3147, 0.5477, 0.7071, 0.9989, 1.2237, 1.4141, 1.5811])

Public  tanh: tensor([0.0997, 0.2913, 0.4621, 0.7616, 0.9051, 0.9640, 0.9866])
Private tanh: tensor([0.0994, 0.2914, 0.4636, 0.7636, 0.9069, 0.9652, 0.9873])


## Control Flow using Encrypted Tensors

Note that ```CrypTensors``` cannot be used directly in conditional expressions. Because the tensor is encrypted, the boolean expression cannot be evaluated unless the tensor is decrypted first. Attempting to execute control flow using an encrypted condition will result in an error.

Some control flow can still be executed without decrypting, but must be executed using mathematical expressions. We have provided the function ```crypten.where(condition, x, y)``` to abstract this kind of conditional value setting.

The following example illustrates how to write this kind conditional logic for ```CrypTensors```.

In [6]:
x_enc = crypten.cryptensor(2.0)
y_enc = crypten.cryptensor(4.0)

a, b = 2, 3

# Normal Control-flow code will raise an error
try:
    if x_enc < y_enc:
        z = a
    else:
        z = b
except RuntimeError as error:
    print(f"RuntimeError caught: \"{error}\"\n")

    
# Instead use a mathematical expression
use_a = (x_enc < y_enc)
z_enc = use_a * a + (1 - use_a) * b
print("z:", z_enc.get_plain_text())
    
    
# Or use the `where` function
z_enc = crypten.where(x_enc < y_enc, a, b)
print("z:", z_enc.get_plain_text())

RuntimeError caught: "Cannot evaluate MPCTensors to boolean values"

z: tensor(2.)
z: tensor(2.)


### Advanced Indexing
CrypTen supports many of the operations that work on ```torch``` tensors. Encrypted tensors can be indexed, concatenated, stacked, reshaped, etc. For a full list of operations, see the CrypTen documentation.

In [7]:
x_enc = crypten.cryptensor([1.0, 2.0, 3.0])
y_enc = crypten.cryptensor([4.0, 5.0, 6.0])

# Indexing
z_enc = x_enc[:-1]
print("Indexing:\n", z_enc.get_plain_text())

# Concatenation
z_enc = crypten.cat([x_enc, y_enc])
print("\nConcatenation:\n", z_enc.get_plain_text())

# Stacking
z_enc = crypten.stack([x_enc, y_enc])
print('\nStacking:\n', z_enc.get_plain_text())

# Reshaping
w_enc = z_enc.reshape(-1, 6)
print('\nReshaping:\n', w_enc.get_plain_text())

Indexing:
 tensor([1., 2.])

Concatenation:
 tensor([1., 2., 3., 4., 5., 6.])

Stacking:
 tensor([[1., 2., 3.],
        [4., 5., 6.]])

Reshaping:
 tensor([[1., 2., 3., 4., 5., 6.]])



### Implementation Note

Due to internal implementation details, ```CrypTensors``` must be the first operand of operations that combine ```CrypTensors``` and ```torch``` tensors. That is, for a ```CrypTensor``` ```x_enc``` and a plaintext tensor ```y```:
- The expression ```x_enc < y``` is valid, but the equivalent expression ```y > x_enc``` will result in an error.
- The expression ```x_enc + y``` is valid, but the equivalent expression ```y + x_enc``` will result in an error.

We intend to add support for both expressions in the future.